In [ ]:
from caffe2.python import workspace
from detectron.core.config import assert_and_infer_cfg
from detectron.core.config import cfg
from detectron.core.config import merge_cfg_from_file
from detectron.utils.io import cache_url
from detectron.utils.logging import setup_logging
from detectron.utils.timer import Timer
import detectron.core.test_engine as infer_engine
import detectron.datasets.dummy_datasets as dummy_datasets
import detectron.utils.c2 as c2_utils
import detectron.utils.vis as vis_utils

import pycocotools.mask as mask_util
import cv2
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.pylab as pylab

In [ ]:
# this makes our figures bigger
pylab.rcParams['figure.figsize'] = 20, 12
def imshow(img):
    plt.imshow(img[:, :, [2, 1, 0]])
    plt.axis("off")

In [ ]:
c2_utils.import_detectron_ops()
# OpenCL may be enabled by default in OpenCV3; disable it because it's not
# thread safe and causes unwanted GPU memory allocations.
cv2.ocl.setUseOpenCL(False)

In [ ]:
CONFIG_FILE = "/opt/DensePose/configs/DensePoseKeyPointsMask_ResNet50_FPN_s1x-e2e.yaml"
MODEL_FILE = "https://dl.fbaipublicfiles.com/densepose/DensePoseKeyPointsMask_ResNet50_FPN_s1x-e2e.pkl"

merge_cfg_from_file(CONFIG_FILE)
cfg.TEST.NMS = 0.3
cfg.NUM_GPUS = 1
model_weights = cache_url(MODEL_FILE, cfg.DOWNLOAD_CACHE)
assert_and_infer_cfg(cache_urls=False)
model = infer_engine.initialize_model_from_cfg(model_weights)

In [ ]:
image = cv2.imread('../DensePoseData/demo_data/input3.png')

In [ ]:
with c2_utils.NamedCudaScope(0):
    cls_boxes, cls_segms, cls_keyps, cls_bodys = infer_engine.im_detect_all(
        model, image, None, None
    )

In [ ]:
result = vis_utils.vis_one_image_opencv(image, cls_boxes, cls_segms, cls_keyps, show_box=True)
imshow(result)